In [11]:
from PIL import Image

import json
import cv2 
import os
import glob 
import re
from os import path


import pickle
import pandas as pd
import numpy as np
import PIL.Image
import time
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
%matplotlib inline 





In [12]:
#reading from json file
#img_dir = "F:/CSE465_Bangla_OCR/out_50000"
#data_path = os.path.join(img_dir,'*g')
#files = glob.glob(data_path) 
#print(files)
file=[]
word_list=[]
data = {}
data['shape'] = []
label_dict={}

#######################################################################
#for i in range(len(files)):
#    file.append(files[i])
#    #print(file[0])
#    inp_file=str(file[0])
#    #print(inp_file)
#    inp_file = inp_file.lstrip('F:/CSE465_Bangla_OCR/out_50000\ ')
#    inp_file = inp_file.rstrip('.jpg')
#    inp_file = inp_file.lstrip('0123456789_')
#    
#    word_list.append(inp_file)
    
    
#    file=[]
    
###########################################################################

data_file = open("word_dict.pkl", "rb")
word_list = pickle.load(data_file)

data_file.close()


#########################################################
#data_file = open("word_dict.pkl", "wb")
#pickle.dump(word_list, data_file)
#data_file.close()    
##########################################################


In [13]:
def extract_graphemes(word):
    support_chars = ['্', 'ং', 'ঃ', 'ঁ', 'ি', 'ু', 'ূ', 'ৃ', 'ে', 'ো', 'ৌ' ,'ী', 'া', 'ে', 'ৈ']
    ref_chars = [ '্য', '্র', 'র্', 'য', 'র']
    unicode_garbage = ['\x02', '\x03', '\x06', '\x08', '\x10', '\x12', '&', '¡',
                        '¤', '¥', '¦', '©', '¬', '\xad', '®', '¯', 'Ä', 'Í', 'ä', 'æ', 'è', 'ø', 'ÿ',
                        'œ', 'š', 'Ÿ', 'ƒ', 'β', '॥', '\u09e4', '\u200b', '\u200d', '\u200f', '\uf020',
                        '\uf02d', '�', '\u200b', '\u200c', '\u09e5']
    
    chars = []
    i = 0
    prev_ref = False

    while(i < len(word)):
        if word[i] != support_chars[0] and word[i] not in unicode_garbage:
            if i+1 < len(word):
                if word[i+1] != support_chars[0]:
                    if word[i+1] == ref_chars[-1] and i+2 < len(word):
                        if word[i+2] == support_chars[0]:
                            chars.append(word[i])
                            chars.append(ref_chars[2])
                            i += 2
                            prev_ref = True
                        else:
                            chars.append(word[i])
                            i += 1
                    else:
                        chars.append(word[i])
                        i += 1
                elif word[i+1] == support_chars[0] and word[i] not in support_chars[0:]:
                    
                    previous = False
                    isSupport = True
                    idx = i+1
                    if idx<len(word):
                        while(isSupport):
                            if idx<len(word):
                                #print(word[i], word[idx], i, idx)
                                if (word[idx] == support_chars[0] or word[idx] == ref_chars[4]) and idx+1 < len(word):
                                    if word[idx] == support_chars[0] and word[idx-1] == ref_chars[-1]:
                                        if not previous:
                                            if i != idx:
                                                chars.append(word[i:(idx-1)])
                                            chars.append(ref_chars[2])
                                            idx += 1
                                            i = idx
                                            continue
                                    if word[idx] == ref_chars[-1]:

                                        if word[idx+1] != support_chars[0]:
                                            chars.append(ref_chars[-1])
                                        idx += 1
                                        i = idx
                                        continue
                                    if word[idx+1] == ref_chars[3]:
                                        if i != idx:
                                            chars.append(word[i:idx])
                                        chars.append(ref_chars[0])
                                        idx += 2
                                        i = idx
                                        # print(i)
                                        # print(idx)
                                        continue
                                    if word[idx+1] == ref_chars[4]:
                                        # print(chars)
                                        if i != idx:
                                            chars.append(word[i:idx])
                                        chars.append(ref_chars[1])
                                        idx += 2
                                        i = idx
                                        previous = True
                                        continue
                                    if word[idx+1] == '\u200c':
                                        if i != idx:
                                            chars.append(word[i:idx])
                                        i = idx+2
                                        isSupport = False

                                    idx += 2
                                else:
                                    isSupport= False
                            else:
                                isSupport = False
                    if i != idx:
                        chars.append(word[i:idx])
                    i = idx
                else:
                    if word[i] in support_chars[0:]:
                        chars.append(word[i])
                    i += 2
            else:
                chars.append(word[i])
                i += 1
        else:
            if word[i]== support_chars[0]:
                if prev_ref:
                    prev_ref = False
                    i += 1
                    continue
                chars.append(word[i])
                i+=1
                continue
            else:
                i+=1
                continue
        
    return chars

In [14]:
def concet(word):
    sborno=['\u0985','\u0986','\u0987','\u0988','\u0989','\u098A','\u098B','\u098F','\u0990','\u0993','\u0994']
    bborno =['ক','খ','গ','ঘ','ঙ','চ','ছ','জ','ঝ','ঞ','ট','ঠ','ড','ঢ','ণ','ত','থ','দ','ধ','ন','প','ফ','ব','ভ','ম','য','র','ল','শ','ষ','স','হ','ৎ','ড়','ঢ়','য়','০','১','২','৩','৪','৫','৬','৭','৮','৯']
             
    support_chars = ['্', 'ং', 'ঃ', 'ঁ', 'ি', 'ু', 'ূ', 'ৃ', 'ে', 'ো', 'ৌ' ,'ী', 'া', 'ে', 'ৈ']
    ref_chars = [ '্য', '্র', 'র্', 'য', 'র','়']
    unicode_garbage = ['\x02', '\x03', '\x06', '\x08', '\x10', '\x12', '&', '¡',
                    '¤', '¥', '¦', '©', '¬', '\xad', '®', '¯', 'Ä', 'Í', 'ä', 'æ', 'è', 'ø', 'ÿ',
                    'œ', 'š', 'Ÿ', 'ƒ', 'β', '॥', '\u09e4', '\u200b', '\u200d', '\u200f', '\uf020',
                    '\uf02d', '�', '\u200b', '\u200c', '\u09e5']



    con=[]
    refF=False
    i=0
    counter=0
    totcount=0
    temp=0
    while i<(len(word)-1):
        refF=False
        #print(i)
        for sc in range(len(support_chars)):
            if(word[i+1]== support_chars[sc]):
                refF=True
                break
        for rc in range(len(ref_chars)):
            if(word[i+1]==ref_chars[rc]):
                refF=True
                break
    
    
        for s in range(len(sborno)):
            #if(word[i+1]!= sborno[s]):
             #   refF=True
            if(word[i+1]== sborno[s]):
                refF=False
                break
        for b in range(len(bborno)):
            #if(word[i+1]!=bborno[b]):
             #   refF=True
            if(word[i+1]==bborno[b]):
                refF=False
                break
        #print(refF)
        if(refF==True):
            counter+=1
            totcount+=1
            i=i+1
            if(i==(len(word)-1)):
                if(counter>0):
                    if(counter==1):
                        con.append(word[i-counter]+word[i+1-counter])
                    if(counter==2):
                        con.append(word[i-counter]+word[i+1-counter]+word[i+2-counter])
                    if(counter==3):
                        con.append(word[i-counter]+word[i+1-counter]+word[i+2-counter]+word[i+3-counter])
                    if(counter==4):
                        con.append(word[i-counter]+word[i+1-counter]+word[i+2-counter]+word[i+3-counter]+word[i+4-counter])
        if(refF==False):
            #con.append(word[i-counter:i])
            if(counter>0):
                if(counter==1):
                    con.append(word[i-counter]+word[i+1-counter])
                if(counter==2):
                    con.append(word[i-counter]+word[i+1-counter]+word[i+2-counter])
                if(counter==3):
                    con.append(word[i-counter]+word[i+1-counter]+word[i+2-counter]+word[i+3-counter])
                if(counter==4):
                    con.append(word[i-counter]+word[i+1-counter]+word[i+2-counter]+word[i+3-counter]+word[i+4-counter])
            else:
                con.append(word[i])
            counter=0
            i+=1
            
    wordlen=len(word)
    conlen=len(con)

    if(wordlen-(conlen+totcount)==1):
        con.append(word[len(word)-1])
        
    return con

In [15]:
grapheme_dict = {}
labels = []
words = []
lengths = []
count = 2
#word_list = sorted(word_list, key=lambda x: int(x.split('_')[0]))
for p in range(len(word_list)):
    curr_word = word_list[p]
    #curr_word = name.split('_')[1][:-4]
    curr_label=[]
    grapheme =extract_graphemes(curr_word)
    graphemes = concet(grapheme)
    if 'স্ক্র্' in graphemes:
        print(curr_word)
        print(graphemes)
    for grapheme in graphemes:
        if grapheme not in grapheme_dict:
            grapheme_dict[grapheme] = count
            curr_label.append(count)
            count += 1
        else:
            curr_label.append(grapheme_dict[grapheme])
    lengths.append(len(curr_label))
    labels.append(curr_label)
    

    inv_grapheme_dict = {v: k for k, v in grapheme_dict.items()}
print(grapheme_dict)

{'শা': 2, 'ন্তি': 3, 'চু': 4, 'ক্তি': 5, 'বি': 6, 'রো': 7, 'ধী': 8, 'বা': 9, 'সা': 10, 'ই': 11, 'ল': 12, 'নির্': 13, 'মা': 14, 'ণ': 15, 'কৌ': 16, 'শ': 17, 'আ': 18, 'হী': 19, 'দে': 20, 'র': 21, 'জো': 22, 'গা': 23, 'ড়ে': 24, 'হ্রা': 25, 'সে': 26, 'সর্': 27, 'ব': 28, 'স': 29, 'ম্ম': 30, 'তি': 31, 'ক্র': 32, 'মে': 33, 'টি': 34, 'ক': 35, 'ছা': 36, 'ত্র': 37, 'ও': 38, 'গু': 39, 'লো': 40, 'চি': 41, 'ভা': 42, 'ষ': 43, 'রা': 44, 'জা': 45, 'কা': 46, 'ভূ': 47, 'মি': 48, 'ঊর্': 49, 'ধ্বা': 50, 'শে': 51, 'অ': 52, 'শো': 53, 'ন': 54, 'লে': 55, 'তু': 56, 'সং': 57, 'গ্ন': 58, 'নি': 59, 'ট্্': 60, 'চে': 61, 'পু': 62, 'ঞ্জি': 63, 'ম্প্র': 64, 'দা': 65, 'য়ি': 66, 'তা': 67, 'দো': 68, 'লি': 69, 'শি': 70, 'গ': 71, 'ঞ্জ': 72, 'উ': 73, 'চ্চা': 74, 'ঙ্ক্ষী': 75, 'ম': 76, 'ণ্ড': 77, 'প': 78, 'ফা': 79, 'হ': 80, 'কি': 81, 'ৎ': 82, 'ক্রা': 83, 'ন্ত': 84, 'ব্যাং': 85, 'কে': 86, 'নৌ': 87, 'ধি': 88, 'বর্': 89, 'প্র': 90, 'ষ্ঠা': 91, 'জি': 92, 'লা': 93, 'পি': 94, 'রূ': 95, 'রী': 96, 'ধ্র': 97, 'ড়': 98, 'গো': 99, 'য়

In [16]:

a_file = open("grapheme_dict.pkl", "wb")
pickle.dump(grapheme_dict, a_file)
a_file.close()

In [17]:
lbl_file = open("labels.pkl", "wb")
pickle.dump(labels,lbl_file)
lbl_file.close()

In [18]:
#train_df= pd.read_csv('C:/Users/monsur/Desktop/GraphemeGenerator/dataset/train.csv')
#display(train_df)

In [19]:
#grapheme=[]
#train_id=[]
#for p in range(100):
#    grapheme=(train_df.iloc[p]['grapheme'])
#    train_id = (train_df.iloc[p]['image_id'])
#    for i,name in enumerate(grapheme_dict):
#        if(grapheme ==grapheme_dict[name]):
#            print(grapheme+"=="+grapheme_dict[name])
#    ##now all we need id to compare and match the graphemes with the given characters.
#    print(grapheme)
#    print(train_id)

In [20]:
#for i,name in enumerate(grapheme_dict):
#    print(name)

In [21]:
a_file = open("traingrapheme_dict.pkl", "rb")
prevgrapheme = pickle.load(a_file)

a_file.close()

print(prevgrapheme)

{'জা': 2, 'য়': 3, 'গা': 4, 'ই': 5, 'হা': 6, 'ড়া': 7, 'ভা': 8, 'ঙ্গা': 9, 'মে': 10, 'ল': 11, 'তে': 12, 'দৌঁ': 13, 'ড়ে': 14, 'ক্যা': 15, 'ব': 16, 'লে': 17, 'সর্': 18, 'কে': 19, '৷': 20, '২': 21, '১': 22, '৪': 23, '৩': 24, 'আ': 25, 'ভঁ': 26, 'রা': 27, 'ঙা': 28, 'মা': 29, 'টি': 30, 'য়া': 31, 'র্': 32, 'ণ': 33, 'কু': 34, 'রী': 35, 'সা': 36, 'নো': 37, 'র': 38, 'সি': 39, 'জ': 40, 'ন': 41, 'স': 42, 'রো': 43, 'দ': 44, 'ফি': 45, 'ট': 46, 'নে': 47, 'হী': 48, '০': 49, '৬': 50, '৯': 51, 'কি': 52, 'লা': 53, 'চি': 54, 'ত্র': 55, 'চর্': 56, 'চা': 57, 'তো': 58, 'লো': 59, 'আর্': 60, 'যা': 61, 'বর্': 62, 'ত': 63, 'দ্রী': 64, 'জ্ঞা': 65, 'ন্ডা': 66, 'রে': 67, 'ভি': 68, 'ন্ন': 69, 'ধ': 70, 'এ': 71, 'ডি': 72, 'ম': 73, 'হ': 74, 'ডো': 75, 'ন্ট': 76, 'ন্ড': 77, 'লি': 78, 'পা': 79, 'সূ': 80, 'হং': 81, 'কং': 82, 'বা': 83, 'সী': 84, 'দে': 85, 'ক্ষ': 86, 'তা': 87, 'ম্প': 88, 'জী': 89, 'অ': 90, 'নু': 91, 'ও': 92, 'ড': 93, 'না': 94, 'বে': 95, 'জি': 96, 'স্বী': 97, 'কা': 98, 'মো': 99, 'ম্ম': 100, 'গে': 101, 'য়ে'

In [22]:
b_file = open("uniquetraingrapheme.pkl", "rb")
nowgrapheme = pickle.load(b_file)

b_file.close()

print(nowgrapheme)

{'আ': 2, 'ই': 3, 'ন': 4, 'জী': 5, 'বি': 6, 'কে': 7, 'বা': 8, 'জা': 9, 'য়': 10, '৫': 11, '০': 12, 'টা': 13, 'প্র': 14, 'তি': 15, 'নি': 16, 'ধি': 17, 'ত্ম': 18, 'চা': 19, 'রে': 20, 'ডে': 21, 'ভি': 22, 'ল': 23, 'মা': 24, 'মো': 25, 'প্রে': 26, 'ক্ষা': 27, 'প': 28, 'টে': 29, 'ও': 30, 'প্যা': 31, 'র': 32, 'স': 33, 'খো': 34, 'লা': 35, 'চু': 36, 'গূ': 37, 'ঢ়': 38, 'ক': 39, 'মি': 40, 'শে': 41, 'উ': 42, 'দ্দি': 43, 'নে': 44, 'দুর্': 45, 'ঘ': 46, 'ট': 47, 'না': 48, 'গি': 49, 'য়ে': 50, 'ছি': 51, 'দ': 52, 'রি': 53, 'ত': 54, 'লে': 55, 'খা': 56, 'দ্য': 57, 'সি': 58, 'কা': 59, 'স্টি': 60, 'ভে': 61, 'ন্স': 62, 'হ': 63, 'ড়ি': 64, 'অ': 65, 'তী': 66, 'সাঁ': 67, 'জি': 68, 'রা': 69, 'জে': 70, 'ফা': 71, 'দু': 72, 'টো': 73, 'সা': 74, 'প্রা': 75, 'জিং': 76, 'টি': 77, 'অ্যা': 78, 'ন্ড': 79, 'লো': 80, 'হি': 81, 'শি': 82, 'লি': 83, 'মে': 84, 'ষ': 85, 'ভা': 86, 'গ': 87, 'ভ': 88, 'য়ং': 89, 'ন্দ': 90, 'ন্ধ্যা': 91, 'ডা': 92, 'হু': 93, 'ট্রা': 94, 'স্ট': 95, 'গী': 96, 'ফু': 97, 'লিং': 98, 'ফো': 99, 'নো': 100, 'গ